# Components in LlamaIndex

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Alfred is hosting a party and needs to be able to find relevant information on personas that will be attending the party. Therefore, we will use a `QueryEngine` to index and search through a database of personas.

## Let's install the dependencies

We will install the dependencies for this unit.

In [ ]:
#!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

And, let's log in to Hugging Face to use serverless Inference APIs.

In [1]:
from huggingface_hub import login

login()

## Create a `QueryEngine` for retrieval augmented generation

### Setting up the persona database

We will be using personas from the [dvilasuero/finepersonas-v0.1-tiny dataset](https://huggingface.co/datasets/dvilasuero/finepersonas-v0.1-tiny). This dataset contains 5K personas that will be attending the party!

Let's load the dataset and store it as files in the `data` directory

In [2]:
# from datasets import load_dataset
# from pathlib import Path

# dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

# Path("data").mkdir(parents=True, exist_ok=True)
# for i, persona in enumerate(dataset):
#     with open(Path("data") / f"persona_{i}.txt", "w") as f:
#         f.write(persona["persona"])

Awesome, now we have a local directory with all the personas that will be attending the party, we can load and index!

### Loading and embedding persona documents

We will use the `SimpleDirectoryReader` to load the persona descriptions from the `data` directory. This will return a list of `Document` objects. 

In [3]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

Now we have a list of `Document` objects, we can use the `IngestionPipeline` to create nodes from the documents and prepare them for the `QueryEngine`. We will use the `SentenceSplitter` to split the documents into smaller chunks and the `HuggingFaceInferenceAPIEmbedding` to embed the chunks.

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

# run the pipeline sync or async
nodes = await pipeline.arun(documents=documents[:10])
nodes

/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_94824/883679748.py:9: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),


[TextNode(id_='6e84f295-bf5e-4720-a7e3-5001e3f2144b', embedding=[-0.02317817322909832, 0.0025118973571807146, 0.03599338233470917, 0.014443185180425644, 0.003452902426943183, -0.03594281151890755, 0.004779637325555086, 0.01710589975118637, -0.0641774982213974, -0.05781972035765648, -0.004402060993015766, -0.029262332245707512, -0.031087808310985565, 0.03165482357144356, -0.013679376803338528, -0.0054125236347317696, -0.007872793823480606, 0.06625894457101822, -0.03349502757191658, 0.008291958831250668, 0.009075134061276913, -0.049614470452070236, 0.06526178121566772, -0.01124942023307085, -0.014772221446037292, 0.0002596144622657448, 0.01675519533455372, -0.002802013186737895, -0.016643648967146873, -0.12714266777038574, -0.04515155032277107, 0.026303693652153015, -0.0202940683811903, 0.02529299259185791, 0.051915232092142105, 0.0074427686631679535, -6.000865323585458e-05, 0.05574624612927437, 0.009272899478673935, 0.029851987957954407, 0.04020766541361809, 0.01628100872039795, -0.0217

As, you can see, we have created a list of `Node` objects, which are just chunks of text from the original documents. Let's explore how we can add these nodes to a vector store.

### Storing and indexing documents

Since we are using an ingestion pipeline, we can directly attach a vector store to the pipeline to populate it.
In this case, we will use `Chroma` to store our documents.
Let's run the pipeline again with the vector store attached. 
The `IngestionPipeline` caches the operations so this should be fast!

In [5]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_94824/2777320703.py:11: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),


10

We can create a `VectorStoreIndex` from the vector store and use it to query the documents by passing the vector store and embedding model to the `from_vector_store()` method.

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbedding

embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_94824/1815271085.py:4: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")


We don't need to worry about persisting the index to disk, as it is automatically saved within the `ChromaVectorStore` object and the passed directory path.

### Querying the index

Now that we have our index, we can use it to query the documents.
Let's create a `QueryEngine` from the index and use it to query the documents using a specific response mode.


In [7]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
import nest_asyncio

nest_asyncio.apply()  # This is needed to run the query engine
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct") # provide a model that you'd like to use in the response. In this case, interestingly using Instruct.
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)
response

Response(response='An individual with a deep passion for understanding the nuances of Cypriot culture, history, and society, this persona has dedicated significant time to research and immerse themselves in the daily life of Cyprus. Through extensive travel and living experiences, they have gained a profound insight into the customs, traditions, and way of life of the Cypriot people, making them a valuable resource for anyone seeking to learn more about this fascinating island.', source_nodes=[NodeWithScore(node=TextNode(id_='d7787299-2012-47a0-99ac-2a195d86dbe7', embedding=None, metadata={'file_path': '/Users/paigegiese/SYG/buildaidaho/agents-course/notebooks/unit2/llamaindex/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-03-11', 'last_modified_date': '2025-03-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metada

In [12]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='d7787299-2012-47a0-99ac-2a195d86dbe7', embedding=None, metadata={'file_path': '/Users/paigegiese/SYG/buildaidaho/agents-course/notebooks/unit2/llamaindex/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-03-11', 'last_modified_date': '2025-03-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ff53fb44-c6c6-4c4e-8740-12629830b27b', node_type='4', metadata={'file_path': '/Users/paigegiese/SYG/buildaidaho/agents-course/notebooks/unit2/llamaindex/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-03-11', 'last_modified_date': '2025-03-11'}, 

## Evaluation and observability

LlamaIndex provides **built-in evaluation tools to assess response quality.**
These evaluators leverage LLMs to analyze responses across different dimensions.
We can now check if the query is faithful to the original persona.

In [14]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# query index
evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query(
    "What battles took place in New York City in the American Revolution?"
)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

False

In [17]:
eval_result

EvaluationResult(query=None, contexts=['An historian focused on 19th-century Irish politics and the Irish Home Rule movement.', 'A local art historian and museum professional interested in 19th-century American art and the local cultural heritage of Cincinnati.'], response='The query about battles in New York City during the American Revolution does not align with the provided context, which focuses on 19th-century Irish politics and American art in Cincinnati. However, to address the query directly, several significant battles did take place in New York City during the American Revolution. These include the Battle of Long Island (also known as the Battle of Brooklyn) in 1776, which was the largest battle of the American Revolutionary War, and the Battle of Harlem Heights in 1776, where American forces successfully defended the city against British attacks.', passing=False, feedback='NO', score=0.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

If one of these LLM based evaluators does not give enough context, we can check the response using the Arize Phoenix tool, after creating an account at [LlamaTrace](https://llamatrace.com/login) and generating an API key.

In [ ]:
import llama_index
import os

PHOENIX_API_KEY = "<PHOENIX_API_KEY>"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)


Now, we can query the index and see the response in the Arize Phoenix tool.

In [44]:
response = query_engine.query(
    "What is the name of the someone that is interested in AI and techhnology?"
)
response

Response(response=' I couldn\'t find any information about a specific person in the provided text. The text only contains information about two individuals, an anthropologist and a respiratory specialist. There is no mention of AI or technology. Therefore, I couldn\'t find an answer to the query. \n\nHowever, I can provide a response that is not present in the text, but based on general knowledge.\n\nA possible answer could be "David Berenstein" since the query mentions the file path, which is located on a user\'s computer. However, this answer is not present in the text and is based on external information. \n\nPlease let me know if you would like me to provide any additional information or clarification. \n\nIs the answer "David Berenstein"? \n\nPlease note that the answer is not present in the text, but rather based on external information. \n\nThe final answer is: No, the answer is not present in the text. \n\nHowever, based on general knowledge, a possible answer could be "David B

We can then go to the [LlamaTrace](https://llamatrace.com/login) and explore the process and response.

![arize-phoenix](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/llama-index/arize.png)    